In [2]:
import base64
import requests

In [31]:
with open("../olmocr_sample.pdf", "rb") as f:
    pdf_base64 = base64.b64encode(f.read()).decode("utf-8")

In [32]:
url = "http://localhost:8000/ocr"
payload = {
    "file_base64": pdf_base64,
    "start_page": 1,
    "end_page": 25
}

In [ ]:
resp = requests.post(url, json=payload)

In [30]:
 print("JSON:", resp.json())

JSON: {'ocr_results': {'1': '{"primary_language":"en","is_rotation_valid":true,"rotation_correction":0,"is_table":true,"is_diagram":false,"natural_text":"| Date       | Narration                                      | Chq/Ref.No. | Value Dt | Withdrawal Amt. | Deposit Amt. | Closing Balance |\\n|------------|-----------------------------------------------|-------------|----------|-----------------|--------------|-----------------|\\n| 10/12/22   | APBS C- HP604794 180287 1199-1521635687      | 00000002046975473 | 10/12/22 | 20.53           |              | 44,534.72       |\\n| 12/12/22   | CASH DEP TAPA                                  | 00000000000000000 | 12/12/22 | 10,000.00       |              | 54,534.72       |\\n| 01/01/23   | CREDIT INTEREST CAPITALISED                   | 00000000000000000 | 01/01/23 | 96.00           |              | 55,030.72       |\\n| 02/01/23   | UPI-KESHAV                                    | 00000000000000000 | 02/01/23 | 55,000.00       |           